In [1]:
%pip install Sastrawi

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df_review = pd.read_csv('train(BL).csv')

In [3]:
df_review.head()

,id,header_review,review_sangat_singkat,label
0,0,Mantap Barang Sesuai Pesanan,mantap barang sesuai pesanan,1
1,1,Mantaps.....,Mantaps.....,1
2,2,Barang Sesuai,Terima kasih buka lapaklapak barang sesuai den...,1
3,3,Barang Asli,Barang asli,1
4,4,Mmc 32 Gb. Samsung,Packing rapih barang sesuai pesanan dan ce...,1


In [4]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96329 entries, 0 to 96328
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     96329 non-null  int64 
 1   header_review          96322 non-null  object
 2   review_sangat_singkat  96329 non-null  object
 3   label                  96329 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.9+ MB


In [5]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
sf = StemmerFactory()
stemmer = sf.create_stemmer()
all_stopwords = stopwords.words('indonesian')
corpus = []
for i in range(30000):
  review = re.sub('[^a-zA-Z]', ' ', df_review['review_sangat_singkat'][i])
  review = review.lower()
  review = review.split()
  review = [word for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  review = stemmer.stem(review)
  corpus.append(review)
  

In [7]:
print(corpus)

['mantap barang sesuai pesan', 'mantaps', 'terima kasih buka lapaklapak barang sesuai muas', 'barang asli', 'packing rapih barang sesuai pesan cepat', 'bagus', 'keren bgt sepatu nya nyaman kaki ringan pokok nya muantap', 'nyoba mudah mudah ampuh', 'coba nich mudah an bagus awet', 'g muat d pakek g melar', 'barang sesuai gambar tp bahan kasar tp lumayan harga segitu', 'yah lumayan', 'barang bagus sesuai pesan', 'terima', 'murah banget sesuai order', 'tdk cocok paksa anak', 'brngnya ga bsa dpke cas ga penuh', 'ukur pas harga murah kualitas oke', 'kali beli sni alhamdulilkah muas thank ya kak', 'suka', 'tks gan barang sesuai pesan cepat admin toko fast respon mantep pokok sukses gan', 'bagus', 'flash sale murah murah murah', 'nursalam bel sesuai', 'jos jos jos jos jos jos jos jos jos jos jos jos jos jos jos', 'gamis katun jepangnya bagus pas an dpakai d histori jual bilang', 'bagus sesuai', 'coba konek tampil layar mudah mudah awet', 'bagus ka thanks', 'suka keren bagus bagus kalo bayar',

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vector = TfidfVectorizer()

In [9]:
X = vector.fit_transform(corpus).toarray()
y = df_review.iloc[0:len(X), -1].values

In [10]:
#from sklearn.feature_extraction.text import CountVectorizer
#cv = CountVectorizer(max_features = 1500)
#X = cv.fit_transform(test).toarray()
#y = df_review.iloc[0:len(X), -1].values

In [11]:
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
print(y)

[1 1 1 ... 1 1 1]


In [13]:
len(X)

30000

In [14]:
len(y)

30000

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [16]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [17]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test .reshape(len(y_test), 1)), 1))

[[0 1]
 [0 1]
 [1 1]
 ...
 [0 1]
 [0 1]
 [1 1]]


In [18]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
# Evaluate the performance with important metrics
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, pos_label = 1)
print('Confusion Matrix:\n', cm)
print('=========================================')
print('Accuracy Score:\n', acc)
print('=========================================')
print('F1 Score:\n', f1)
print('=========================================')
print('Classification Report:\n', classification_report(y_test, y_pred, target_names = ['Class 0', 'Class 1']))
print('=========================================')

Confusion Matrix:
 [[ 884  269]
 [5660 2187]]
Accuracy Score:
 0.3412222222222222
F1 Score:
 0.42453654275453756
Classification Report:
               precision    recall  f1-score   support

     Class 0       0.14      0.77      0.23      1153
     Class 1       0.89      0.28      0.42      7847

    accuracy                           0.34      9000
   macro avg       0.51      0.52      0.33      9000
weighted avg       0.79      0.34      0.40      9000

